In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim

In [13]:
!pip3 install hanziconv

     |████████████████████████████████| 276 kB 941 kB/s eta 0:00:01
  Created wheel for hanziconv: filename=hanziconv-0.3.2-py2.py3-none-any.whl size=23215 sha256=c1286dc4c0a02a71de771e38ca56eb37a4ece266e63ce19f7a473436699892c5
  Stored in directory: /home/gavin/.cache/pip/wheels/24/bc/5f/95aceeea892d9bf06c29a9effde5908abcefd3fa23244fdaa5
Successfully built hanziconv


In [11]:
def sumMinDistance(a:torch.tensor,b:torch.tensor):
    return float(torch.sum(torch.tensor(minDistance(a,b))))

def minDistance(a:torch.tensor,b:torch.tensor):
    return [float(torch.topk(i,1,largest=False).values) for i in torch.cdist(a, b, p=1)]
a = torch.tensor([[2.,2.],[1.,1.]])
b = torch.tensor([[1.,1.],[9.,9.]])
minDistance(a,b)
sumMinDistance(a,b)

2.0

In [26]:
import torch
torch.manual_seed(123) 

#
x = torch.randn(4, 3)  # 输入的维度是（128，20）
print(f"x:{x}")
print(f"x.shape:{x.shape}")


m = torch.nn.Linear(3, 2)  # 20,30是指维度
print('m.weight.shape:\n ', m.weight.shape)
print('m:\n ', m)
print('m.bias.shape:\n', m.bias.shape)

#
output = m(x)
print('output.shape:\n', output.shape)
print('output.shape:\n', output)



x:tensor([[-0.1115,  0.1204, -0.3696],
        [-0.2404, -1.1969,  0.2093],
        [-0.9724, -0.7550,  0.3239],
        [-0.1085,  0.2103, -0.3908]])
x.shape:torch.Size([4, 3])
m.weight.shape:
  torch.Size([2, 3])
m:
  Linear(in_features=3, out_features=2, bias=True)
output.shape:
 torch.Size([4, 2])
output.shape:
 tensor([[ 0.6388,  0.4820],
        [ 0.2828, -0.4868],
        [ 0.5248, -0.6461],
        [ 0.6600,  0.5359]], grad_fn=<AddmmBackward>)
m.bias:
  Parameter containing:
tensor([0.4929, 0.2757], requires_grad=True)


In [27]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        print(f"log_probs: {log_probs}")
        return(log_probs)
    
#
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


def get_word_vec(word,word_to_ix,device="cpu"):
    context_ids = make_context_vector(word, word_to_ix).to(device)
    assert context_ids.is_cuda is True # returns a boolean
    return model.embeddings(context_ids)

In [28]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
print(f"vocab: {vocab}")

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(f"data:{data[:5]}")

#

print(f"make_context_vector: {make_context_vector(data[0][0], word_to_ix)}")


vocab: {'spells.', 'about', 'The', 'directed', 'we', 'In', 'computer', 'spirits', 'programs', 'processes.', 'We', 'effect,', 'direct', 'People', 'processes', 'computers.', 'manipulate', 'things', 'beings', 'inhabit', 'idea', 'with', 'Computational', 'is', 'As', 'are', 'abstract', 'evolution', 'process.', 'other', 'program.', 'rules', 'that', 'computational', 'called', 'data.', 'conjure', 'the', 'evolve,', 'pattern', 'study', 'a', 'by', 'of', 'they', 'process', 'create', 'to', 'our'}
data:[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
make_context_vector: tensor([10, 25, 47, 40])


In [29]:
#
device = torch.device('cuda:0')
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), embedding_dim=10, context_size=CONTEXT_SIZE*2)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [30]:
get_word_vec([data[0][0][0]],word_to_ix,device)

tensor([[-1.1820, -0.2877, -0.6043,  0.6002, -1.4053, -0.5922, -0.2548,  1.1517,
         -0.0179,  0.4264]], device='cuda:0', grad_fn=<EmbeddingBackward>)

In [31]:
for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in data:
        context_ids = make_context_vector(context, word_to_ix)
        context_ids = context_ids.to(device)
        model.zero_grad()
        log_probs = model(context_ids)
        label = torch.tensor([word_to_ix[target]], dtype=torch.long)
        label = label.to(device)
        loss = loss_function(log_probs, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

log_probs: tensor([[-3.7700, -3.9725, -4.0603, -3.7126, -3.9289, -3.8381, -3.8448, -3.9837,
         -3.8134, -4.0440, -4.4429, -3.9380, -3.9509, -3.6052, -3.4734, -3.9137,
         -3.9037, -3.9043, -3.6820, -3.9674, -3.9501, -3.7005, -3.5671, -4.0591,
         -4.1642, -3.9919, -4.0892, -3.9892, -4.1895, -3.7773, -3.9236, -3.6267,
         -4.0072, -4.1948, -3.7352, -4.0165, -3.8798, -4.0188, -3.7289, -3.8605,
         -3.6237, -4.0066, -3.8480, -4.0072, -4.1079, -3.8847, -3.8256, -4.2916,
         -3.7414]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
log_probs: tensor([[-3.8496, -3.7114, -4.1726, -3.6015, -4.2844, -3.9159, -3.6612, -4.1978,
         -3.7223, -4.0359, -4.1008, -3.8178, -4.1167, -3.7385, -3.4282, -3.8041,
         -3.9076, -4.0146, -3.6651, -4.0360, -3.8842, -3.6953, -3.8526, -3.8788,
         -4.3756, -3.8490, -4.0023, -3.6333, -4.0987, -3.6976, -3.9581, -3.9625,
         -3.8001, -3.8876, -3.8637, -3.9877, -3.9443, -3.9556, -3.7633, -3.9295,
         -3.8462, -3

log_probs: tensor([[-6.1774, -5.2786, -5.6916, -5.0918, -3.9219, -4.8622, -3.6366, -3.9083,
         -4.7523, -5.2652, -6.1809, -6.0230, -5.5873, -5.1279, -3.5234, -4.8052,
         -5.0092, -5.9059, -4.7825, -4.0934, -4.4670, -3.7181, -5.0606, -5.5054,
         -5.6805, -4.7572, -5.0419, -5.7961, -5.3031, -6.0501, -5.2725, -5.2404,
         -4.8821, -1.0194, -3.6289, -4.6488, -4.2283, -3.5359, -4.7402, -2.5920,
         -4.6696, -3.1703, -5.3541, -3.7085, -4.5215, -3.1592, -4.6027, -3.0996,
         -5.8283]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
log_probs: tensor([[-6.1101, -2.6021, -5.1871, -3.9715, -4.8781, -4.0736, -4.9488, -4.8820,
         -5.1619, -4.2550, -6.1702, -3.2640, -4.9576, -3.8934, -4.2833, -4.7730,
         -5.3767, -3.2540, -4.8814, -5.0356, -4.8453, -3.9335, -5.3345, -4.5119,
         -4.9249, -5.2462, -5.8000, -5.1405, -1.1422, -3.0203, -3.7653, -4.2446,
         -4.8970, -5.5138, -5.4916, -4.3878, -4.1948, -4.4769, -4.5665, -4.9942,
         -3.9978, -6

log_probs: tensor([[-10.6764,  -9.5949,  -9.9519,  -9.0876,  -7.2642,  -8.5002,  -6.9964,
          -7.3677,  -8.1010,  -9.5359, -10.5362, -10.7063,  -9.8908,  -8.6262,
          -6.1551,  -7.9584,  -9.0913, -10.7499,  -7.9900,  -6.4541,  -8.6254,
          -7.8452,  -8.9628,  -9.1414,  -9.6848,  -8.1215,  -9.1072, -10.4008,
          -9.5699, -10.3668,  -9.4706,  -9.5515,  -8.3952,  -0.0389,  -6.9103,
          -7.6445,  -5.7668,  -6.2278,  -8.3465,  -4.3891,  -9.0939,  -7.2138,
          -9.7636,  -6.8866,  -7.7154,  -5.4661,  -8.1832,  -6.0659, -10.1714]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
log_probs: tensor([[-10.8720,  -5.0912,  -9.2567,  -7.1959,  -9.2871,  -7.7211,  -9.7802,
          -8.9039,  -9.2200,  -8.0845, -10.9851,  -5.5357,  -9.2658,  -5.5074,
          -7.9195,  -8.1815, -10.1140,  -6.1268,  -8.4263,  -9.0136,  -9.7129,
          -8.7109,  -9.4947,  -7.1069,  -8.6192,  -9.5632, -10.7103,  -9.4697,
          -0.0359,  -5.7259,  -6.8532,  -7.8707,  -8.

log_probs: tensor([[ -9.3673,  -7.4880,  -6.1579,  -7.8523,  -9.5931,  -7.8430,  -6.8587,
         -10.5867,  -7.7244,  -7.7351, -10.6914,  -9.0766,  -7.7297,  -8.6487,
          -7.9843,  -0.0201,  -9.2299,  -7.6687,  -7.1326,  -8.0457,  -8.3954,
          -7.0004,  -8.3135,  -6.3505,  -8.7575,  -7.4132,  -8.0671,  -7.3096,
          -8.2792,  -9.5225,  -6.9443,  -8.8615,  -7.9352,  -8.3771,  -7.7134,
          -8.2093,  -8.5240,  -8.0701, -10.2061,  -8.9346,  -7.6622, -10.8987,
          -6.8812,  -9.0293,  -8.6225,  -7.4106,  -7.3043,  -8.8236,  -9.6303]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
log_probs: tensor([[-10.9878,  -7.9988,  -9.1707,  -9.3576,  -9.7956,  -8.5953,  -9.2084,
          -7.7516,  -8.0631,  -8.9921, -10.3500,  -5.5734, -10.0404,  -7.8310,
          -9.4406,  -9.1103,  -9.5762,  -8.3951,  -7.2509,  -8.8127,  -8.0050,
          -9.3411,  -7.9558,  -9.1407,  -0.0189,  -8.2067,  -6.9283,  -6.4041,
          -9.0512,  -8.3845,  -8.5420,  -8.7326,  -8.

log_probs: tensor([[-10.1208,  -6.2525,  -6.9879,  -7.8635,  -8.5645,  -8.7586,  -7.6013,
          -8.7658,  -8.6134,  -8.5877, -10.3819, -10.1471,  -8.5585,  -7.8751,
         -10.2436,  -7.6695,  -9.5596,  -7.3491,  -8.7437,  -8.0022,  -7.2709,
          -9.1906,  -7.7437,  -0.0182,  -9.0900,  -9.6973,  -7.8960,  -9.6125,
          -7.8950,  -8.1834,  -9.3505,  -8.1995,  -8.9415,  -9.6550,  -7.7669,
          -7.3604,  -8.4415,  -8.2775,  -8.9172,  -8.5545,  -7.5581, -11.3837,
          -7.4412,  -6.9560,  -7.4317,  -8.3520,  -8.3213,  -5.8444, -10.0932]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)
log_probs: tensor([[-11.0259,  -6.4115, -10.4771,  -0.0150,  -8.7933,  -6.7251,  -8.9642,
          -9.8653,  -9.0186, -10.2550, -10.7663,  -9.3011,  -9.6125, -10.0835,
          -9.4914,  -9.1428, -10.9361,  -7.3676,  -7.7646,  -8.6835,  -7.7079,
          -7.3568,  -8.9421,  -7.9668,  -9.8047, -11.4758,  -6.5319,  -7.9191,
          -8.5903,  -9.1129,  -8.1389,  -8.2998, -10.

In [7]:
get_word_vec([data[0][0][0]],word_to_ix,device)


tensor([[ 0.9557, -1.8957,  0.3716, -1.0030,  1.3066,  1.2743,  1.8235,  1.2366,
         -1.4658, -1.0246]], device='cuda:0', grad_fn=<EmbeddingBackward>)

## 中文

In [8]:
!pip3 install jieba


In [9]:
import jieba

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
text = "華航諾富特飯店群聚案新增首例外包商水電工案1145確診，中央流行疫情指揮中心持續追查水電工感染源，衛福部長陳時中與疾管署長周志浩今天上午一同到台大醫院參加「清潔雙手，攜手抗疫」記者會，會前陳時中受訪透露，案1145可能曾到過諾富特飯店一館B1用餐，不排除任何感染可能。指揮中心昨天公佈的案1145，是在諾富特飯店的一館3、5、6樓層（整修樓層）工作，由於和飯店人員、旅客都沒有接觸，因此感染源不明；不過陳時中今天表示，在施工的地方大部分都是工作人員，但是可能有在B1的地方共餐。媒體追問，是否表示案1145有去過B1，是不是可能被案1120或是餐飲部的員工感染？陳時中回應，各種可能性都有，但還需要相關的釐清。"
# raw_text = [t for t in text]
raw_text = list(jieba.cut(text))


# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
print(f"vocab: {vocab}")

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(f"data:{data[:5]}")

#

print(f"make_context_vector: {make_context_vector(data[0][0], word_to_ix)}")

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.360 seconds.
Prefix dict has been built successfully.


vocab: {'水電', '包商', '感染', '華航諾富', '清潔', '樓層', '；', '）', '沒', '受訪', '釐', '媒體', '群聚', '特飯', '6', '都', '不過', '是不是', '特飯店', '1120', '？', '工案', '人員', '案', '。', '排除', '「', '首', '可能', '店', '，', '長', '地方', '疫情', '任何', '不明', '參加', '今天', '一同', '時', '與', '和', '透露', '可能性', '攜手', '是否', '整修', '台大', '昨天', '餐飲部', '員工', '施工', '持續', '1145', '例外', '流行', '因此', '共餐', '疾管署', '用餐', '到', '旅客', '指揮', '今天上午', '中心', '醫院', '表示', '雙手', '3', '接觸', '被案', '周志浩', '需要', '工作', '中', '在', '抗疫', '去過', '公', '回應', '源', '衛福部長', '各種', '」', '的', '但是', '由', '（', '一館', '記者', '大部分', '佈', '曾到', '陳', '新增', '追查', '諾富', '清', '會', '確診', '但還', '水電工', '追問', '5', '中央', '過諾富', '或是', '是', 'B1', '、', '會前', '飯店', '不', '有', '於', '相關'}
data:[(['華航諾富', '特飯', '群聚', '案'], '店'), (['特飯', '店', '案', '新增'], '群聚'), (['店', '群聚', '新增', '首'], '案'), (['群聚', '案', '首', '例外'], '新增'), (['案', '新增', '例外', '包商'], '首')]
make_context_vector: tensor([ 3, 13, 12, 23])


In [10]:
#
embedding_dim_num = 30
device = torch.device('cuda:0')
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), embedding_dim=embedding_dim_num, context_size=CONTEXT_SIZE*2)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [11]:
for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in data:
        context_ids = make_context_vector(context, word_to_ix)
        context_ids = context_ids.to(device)
        model.zero_grad()
        log_probs = model(context_ids)
        label = torch.tensor([word_to_ix[target]], dtype=torch.long)
        label = label.to(device)
        loss = loss_function(log_probs, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

[tensor([854.1604]), tensor([506.0363]), tensor([173.0192]), tensor([117.9374]), tensor([48.5274]), tensor([8.8048]), tensor([0.6025]), tensor([0.3661]), tensor([0.3051]), tensor([0.2642])]


In [12]:
print(f"text:{[data[0][0][0]]}, vec: {get_word_vec([data[0][0][0]],word_to_ix,device)}")

text:['華航諾富'], vec: tensor([[ 0.3344, -0.0270,  0.4334, -0.9581,  1.1527, -1.2934, -1.6101,  0.6169,
         -1.7118,  0.0706, -0.5426, -0.2043,  0.1286, -0.5433, -0.0819,  0.3740,
         -0.8154,  1.0640,  0.0960, -0.2247,  1.5876, -0.4896, -0.7010, -0.5196,
          0.9647,  0.8373,  0.0902, -0.6911, -0.8016,  0.8198]],
       device='cuda:0', grad_fn=<EmbeddingBackward>)


In [13]:
a = get_word_vec([data[0][0][0]],word_to_ix,device)
b = get_word_vec([data[0][0][1]],word_to_ix,device)
torch.cdist(a, b, p=embedding_dim_num)

tensor([[2.4621]], device='cuda:0', grad_fn=<CdistBackward>)

In [14]:
import torch
PATH_MODEL_SAVE = "./save_word2vec.pt"

# Save:
torch.save(model.state_dict(), PATH_MODEL_SAVE)

In [15]:
# Load:
CONTEXT_SIZE = 2
embedding_dim_num =30
PATH_MODEL_SAVE = "./save_word2vec.pt"
device = torch.device('cuda:0')


model = CBOW(len(vocab), embedding_dim=embedding_dim_num, context_size=CONTEXT_SIZE*2)
model.load_state_dict(torch.load(PATH_MODEL_SAVE))
model.to(device)
get_word_vec(["華航諾富"],word_to_ix,device)


tensor([[ 0.3344, -0.0270,  0.4334, -0.9581,  1.1527, -1.2934, -1.6101,  0.6169,
         -1.7118,  0.0706, -0.5426, -0.2043,  0.1286, -0.5433, -0.0819,  0.3740,
         -0.8154,  1.0640,  0.0960, -0.2247,  1.5876, -0.4896, -0.7010, -0.5196,
          0.9647,  0.8373,  0.0902, -0.6911, -0.8016,  0.8198]],
       device='cuda:0', grad_fn=<EmbeddingBackward>)